In [2]:
import numpy as np
import pandas as pd
import pm4py
from scipy import stats
import import_ipynb
import time
import numbers
from timeOperations import calculateCycleTimeOfEachPair
from shapleyAdditiveValues import getFunction

importing Jupyter notebook from timeOperations.ipynb


In [3]:
#add the data to notebook directory
# import os,sys,inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0,parentdir)
# import sys
# sys.path.insert(0, './Basic operations/timeOperations.ipynb')
# from timeOperations import calculateCycleTimeOfEachPair

In [4]:
# log = pm4py.read_xes('DomesticDeclarations.xes_')
# dataframe = pm4py.convert_to_dataframe(log)

In [5]:
#log_ct=calculateCycleTimeOfEachPair(dataframe,"case:id","time:timestamp","concept:name")

In [6]:
#df_agrupado_trazas=log_ct.groupby(['case:id']).agg(lambda x: list(x))
#version cutre para calcular la suma del cycle time para cada fila
#log_ct['CT total']=[sum(log_ct[log_ct['case:id']==case]['Cycle time']) for case in log_ct['case:id']]

In [15]:
def indexNumericalValue(dictionary):
    labels={}
    bins={}
    
    for dimension,KPI in dictionary.items():
        nbinsCase=round(freedman_diaconis(KPI,"number"))
        #widthCaseAmount=round(freedman_diaconis(dataframe["case:Amount"],"width"))
        discretized_case=pd.cut(KPI.values,nbinsCase).value_counts().sort_values(ascending=False).replace(0,np.nan).dropna()
        dictionary[dimension]=(KPI,discretized_case)
        bins[dimension]=nbinsCase
        labels[dimension]=discretized_case.index.categories
    
    return dictionary,labels,bins

In [8]:
def propertiesFunc(x):
    vals={}
    vals['numAct']=len(x['concept:name'])
    vals['totalTime']=sum(x['Cycle time'])
    vals['numRes']=len(x['org:resource'])
    vals['avgRoles']=np.mean(x['org:role'].value_counts())
    
    return pd.Series(vals, index=['numAct', 'totalTime', 'numRes', 'avgRoles'])

In [9]:
#listaOp=log_ct.groupby("case:id").apply(lambda x: len(x['concept:name']))

In [10]:
def freedman_diaconis(data, returnas):
    """
    Use Freedman Diaconis rule to compute optimal histogram bin width. 
    ``return`` can be one of "width" or "bins", indicating whether
    the bin width or number of bins should be returned respectively. 


    Parameters
    ----------
    data: np.ndarray
        One-dimensional array.

    returns: {"width", "bins"}
        If "width", return the estimated width for each histogram bin. 
        If "bins", return the number of bins suggested by rule.
    """
    data = np.asarray(data, dtype=np.float_)
    IQR  = stats.iqr(data, rng=(25, 75), scale="raw", nan_policy="omit")
    N    = data.size
    bw   = (2 * IQR) / np.power(N, 1/3)

    if returnas=="width":
        result = bw
    elif returnas=="number":
        datmin, datmax = data.min(), data.max()
        datrng = datmax - datmin
        result = int((datrng / bw) + 1)
        
    return result

In [11]:
def calculateAggregatedCycleTime(df,attribute,cycle):
    aggregated=df.groupby(by=attribute).apply(lambda x: x[cycle].mean())
    return aggregated

In [ ]:
def calculateFrequency(df,attribute):
    aggregated=df[attribute].value_counts()
    return aggregated

In [1]:
def enrichment(dataframe,caseId,timestamp,activities):
    

    """
    Inputs:
        dataframe:log in dataframe format
        caseId: name of the case column
        timestamp: name of the timestamp column
        activities: name of the column of activities
    
    Outputs:
        log_ct:log enrichened with properties such as cycle time for events, number of activites per trace...etc
    
    """
    #start_time = time.time()
    log_ct=calculateCycleTimeOfEachPair(dataframe,caseId,timestamp,activities)
    log_ct['Cycle time']=log_ct['Cycle time']/60
    #aggregated KPIs over traces
#     listaVals=log_ct.groupby(caseId).apply(lambda x: propertiesFunc(x))
#     listaNumAct=[]
#     listaTotalTime=[]
#     listaNumRes=[]
#     listaAvgRoles=[]

#     for name in log_ct['case:id']:
#         fila=listaVals[listaVals.index.isin([name])]
#         listaNumAct.append(fila['numAct'].values[0])
#         listaTotalTime.append(fila['totalTime'].values[0])
#         listaNumRes.append(fila['numRes'].values[0])
#         listaAvgRoles.append(fila['avgRoles'].values[0])

    #Intento de creación simultanea de varias columnas pero no se como pasar a nuestro el data
    #supongo que sería un vector con los valores por filas 
    #data = np.random.randint(10, size=(5,2))
    #df_add = pd.DataFrame(data=data,columns=['numAct','totalTime','numRes','avgRoles'])
    #log_ct = pd.concat([log_ct,df_add], axis=1)

#     log_ct['numAct']=listaNumAct
#     log_ct['totalTime']=listaTotalTime
#     log_ct['numRes']=listaNumRes
#     log_ct['avgRoles']=listaAvgRoles
#     log_ct.set_index(['case:id','numAct','totalTime','numRes','avgRoles','id'], inplace=True)
    #print("--- %s seconds ---" % (time.time() - start_time))
    
    
    kpi_frequencies_roles=log_ct['org:role'].value_counts()
    kpi_frequencies_resources=log_ct['org:resource'].value_counts()
    kpi_frequencies_activites=log_ct['concept:name'].value_counts()
    kpi_frequencies_organizations=log_ct['case:Permit OrganizationalEntity'].value_counts()
    
    
    discreteKPIs={
          ("calculateFrequency","org:role","-"):kpi_frequencies_roles,
          ("calculateFrequency","org:resource","-"):kpi_frequencies_resources,
          ("calculateFrequency","concept:name","-"):kpi_frequencies_activites,
          ("calculateFrequency","case:Permit OrganizationalEntity","-"):kpi_frequencies_organizations
        
         }
    

    kpi_activities_time=calculateAggregatedCycleTime(log_ct,"concept:name","Cycle time")
    kpi_resources_time=calculateAggregatedCycleTime(log_ct,"org:resource","Cycle time")
    kpi_roles_time=calculateAggregatedCycleTime(log_ct,"org:role","Cycle time")
    kpi_org_time=calculateAggregatedCycleTime(log_ct,'case:Permit OrganizationalEntity',"Cycle time")
    kpi_events_time=log_ct['Cycle time']

    numericKPIs={
          ("calculateAggregatedCycleTime","concept:name","Cycle time"):kpi_activities_time,
          ("calculateAggregatedCycleTime","org:resource","Cycle time"):kpi_resources_time,
          ("calculateAggregatedCycleTime","org:role","Cycle time"):kpi_roles_time,
          ("calculateAggregatedCycleTime","case:Permit OrganizationalEntity","Cycle time"):kpi_org_time,
         }
    
    numericKPIsFixed,labels,bins=indexNumericalValue(numericKPIs)
    
    discreteKPIs.update(numericKPIsFixed)
    
    return log_ct,discreteKPIs,labels,bins

In [ ]:
# kpisDesired={
#     "calculateAggregatedCycleTime":[["concept:name","Cycle time"],["org:role","Cycle time"],[ 'case:Permit OrganizationalEntity','Cycle time']]
#     "calculateFrequency":[["org:role"],["org:resource"],['case:Permit OrganizationalEntity']]
# }
def enrichmentRefined(dataframe,caseId,timestamp,activities,kpisDesired,dispatcherKPI):
    
    """
    Inputs:
        dataframe:log in dataframe format
        caseId: name of the case column
        timestamp: name of the timestamp column
        activities: name of the column of activities
    
    Outputs:
        log_ct:log enrichened with properties such as cycle time for events, number of activites per trace...etc
    
    """
    #start_time = time.time()
    log_ct=calculateCycleTimeOfEachPair(dataframe,caseId,timestamp,activities)
    labels={}
    bins={}
    kpìs={}
    
    #{"calculateFrequency":[["org:role"],["org:resource"],...],calculateAggregatedCycleTime }
    for kpiFunction,setParameters in kpisDesired.items():
        fun=getFunction(kpiFunction,dispatcherKPI)
        for parameters in setParameters:
            res=fun(df,parameters)#calculateFrequency(log_ct,["org:role"]),#calculateFrequency(log_ct,["org:resource"]),
            
            if kpiFunction in kpiNumerics:#if it is a numeric kpi, 
                nbinsCase=round(freedman_diaconis(KPI,"number"))
                discretized_case=pd.cut(KPI.values,nbinsCase).value_counts().sort_values(ascending=False).replace(0,np.nan).dropna()
                res=(res,discre)
                bins[tuple(kpiFuncion,parameters[0],parameters[1])]=nbinsCase
                labels[tuple(kpiFuncion,parameters[0],parameters[1])]=discretized_case.index.categories
            
            if(len(parameters)<2):
                kpis[tuple(kpiFuncion,parameters[0],"-")]=res
            else:
                kpis[tuple(kpiFuncion,parameters[0],parameters[1])]=res
                
    
    return log_ct,kpis,labels,bins

In [28]:
#val_ct_traces=[(case,sum(log_ct[log_ct['case:id']==case]['Cycle time']) ) for case in log_ct['case:id'].unique()]

In [33]:
#log_ct.index.get_level_values('totalTime').unique()